In [ ]:
import batoid
import os
import yaml
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
HSC_fn = os.path.join(batoid.datadir, "HSC", "HSC.yaml")
config = yaml.load(open(HSC_fn))
HSC_telescope = batoid.parse.parse_optic(config['opticalSystem'])

In [ ]:
LSST_fn = os.path.join(batoid.datadir, "LSST", "LSST_r.yaml")
config = yaml.load(open(LSST_fn))
LSST_telescope = batoid.parse.parse_optic(config['opticalSystem'])

In [ ]:
DECam_fn = os.path.join(batoid.datadir, "DECam", "DECam.yaml")
config = yaml.load(open(DECam_fn))
DECam_telescope = batoid.parse.parse_optic(config['opticalSystem'])

In [ ]:
# HSC
max_th = 0.74
pixSize = 15e-6
telescope = HSC_telescope

dthx_drx = []
dthy_dry = []
thetas = np.linspace(0., max_th, 20)
for th in thetas:
    dth_dr = batoid.psf.dthdr(telescope, np.deg2rad(th), 0, 620e-9)
    dthx_drx.append(abs(dth_dr[0,0]))
    dthy_dry.append(abs(dth_dr[1,1]))

dthx_drx = np.array(dthx_drx) * 206265 * pixSize # arcsec per pixel
dthy_dry = np.array(dthy_dry) * 206265 * pixSize # arcsec per pixel

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))
ax1.plot(thetas, dthx_drx, c='b', label='radial')
ax1.plot(thetas, dthy_dry, c='r', label='tangential')
ax1.set_xlabel('radius (deg)')
ax1.set_ylabel('pixel size (arcsec)')
ax1.legend(loc='lower left')
ax1.set_title("HSC")

ax2.plot(thetas, np.array(dthx_drx)/np.array(dthy_dry))
ax2.set_xlabel('radius (deg)')
ax2.set_ylabel('b/a')
ax2.set_title("HSC")

fig.tight_layout()
fig.show()

In [ ]:
# DECam
max_th = 1.1
pixSize = 15e-6
telescope = DECam_telescope

dthx_drx = []
dthy_dry = []
thetas = np.linspace(0., max_th, 20)
for th in thetas:
    dth_dr = batoid.psf.dthdr(telescope, np.deg2rad(th), 0, 620e-9)
    dthx_drx.append(abs(dth_dr[0,0]))
    dthy_dry.append(abs(dth_dr[1,1]))

dthx_drx = np.array(dthx_drx) * 206265 * pixSize # arcsec per pixel
dthy_dry = np.array(dthy_dry) * 206265 * pixSize # arcsec per pixel

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))
ax1.plot(thetas, dthx_drx, c='b', label='radial')
ax1.plot(thetas, dthy_dry, c='r', label='tangential')
ax1.set_xlabel('radius (deg)')
ax1.set_ylabel('pixel size (arcsec)')
ax1.legend(loc='lower left')
ax1.set_title("DECam")

ax2.plot(thetas, np.array(dthx_drx)/np.array(dthy_dry))
ax2.set_xlabel('radius (deg)')
ax2.set_ylabel('b/a')
ax2.set_title("DECam")

fig.tight_layout()
fig.show()

In [ ]:
# LSST
max_th = 1.75
pixSize = 10e-6
telescope = LSST_telescope

dthx_drx = []
dthy_dry = []
thetas = np.linspace(0., max_th, 20)
for th in thetas:
    dth_dr = batoid.psf.dthdr(telescope, np.deg2rad(th), 0, 620e-9)
    dthx_drx.append(abs(dth_dr[0,0]))
    dthy_dry.append(abs(dth_dr[1,1]))

dthx_drx = np.array(dthx_drx) * 206265 * pixSize # arcsec per pixel
dthy_dry = np.array(dthy_dry) * 206265 * pixSize # arcsec per pixel

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 4))
ax1.plot(thetas, dthx_drx, c='b', label='radial')
ax1.plot(thetas, dthy_dry, c='r', label='tangential')
ax1.set_xlabel('radius (deg)')
ax1.set_ylabel('pixel size (arcsec)')
ax1.legend(loc='lower left')
ax1.set_title("LSST")

ax2.plot(thetas, np.array(dthx_drx)/np.array(dthy_dry))
ax2.set_xlabel('radius (deg)')
ax2.set_ylabel('b/a')
ax2.set_title("LSST")

fig.tight_layout()
fig.show()